## Import & Initialize

In [14]:
import os
import time
from datetime import datetime
from dotenv import load_dotenv

import googlemaps
import gmplot

import pandas as pd

from io import StringIO
import uuid
import ast

In [15]:
load_dotenv()
API_KEY = os.getenv('GOOGLE_MAPS_API_KEY_3')

In [16]:
gmaps = googlemaps.Client(key=API_KEY)

In [17]:
df = pd.read_csv('filtered_road_names.csv')
# df.head()

In [19]:
# df.duplicated().sum()

In [20]:
# df.info()

In [21]:
df = df[(df['Distance (km)'] <= 81) & (df['Distance (km)'] >= 7)]

## Code

In [30]:
gmaps = googlemaps.Client(key=API_KEY)
df = pd.read_csv('filtered_road_names.csv')


def determine_traffic_color(delay, duration):
    if delay < 0.05 * duration:
        return 'Blue'
    elif delay < 0.20 * duration:
        return 'Yellow'
    elif delay < 0.50 * duration:
        return 'Orange'
    elif delay < 1.00 * duration:
        return 'Red'
    else:
        return 'Dark Red'

def get_traffic_data_from_dataframe(gmaps_client, start_lat, start_lng, end_lat, end_lng, road_name, city='Riyadh'):
    origin = f"{start_lat},{start_lng}"
    destination = f"{end_lat},{end_lng}"
    
    try:
        directions_result = gmaps_client.directions(
            origin, destination, mode="driving", departure_time="now", traffic_model="best_guess"
        )

        if not directions_result:
            return None

        route = directions_result[0]['legs'][0]
        duration_in_traffic_min = route['duration_in_traffic']['value'] / 60
        distance_km = route['distance']['value'] / 1000
        speed_kmh = distance_km / (duration_in_traffic_min / 60)
        delay_min = (route['duration_in_traffic']['value'] - route['duration']['value']) / 60
        traffic_condition = determine_traffic_color(delay_min, route['duration']['value'] / 60)
        timestamp = datetime.now().isoformat()
        
        return {
            'road_name': road_name,
            'distance_km': distance_km,
            'duration_in_traffic_min': duration_in_traffic_min,
            'speed_kmh': speed_kmh,
            'delay_min': delay_min,
            'traffic_condition': traffic_condition,
            'timestamp': timestamp,
            'Start_point': (start_lat, start_lng),
            'End_point': (end_lat, end_lng),
        }
    except Exception as e:
        print(f"Error fetching traffic data for {road_name}: {e}")
        return None

def generate_data_from_dataframe(csv_file, gmaps_client):
    df = pd.read_csv(csv_file)
    
    traffic_data_list = []
    df['Start_point'] = df['Start_point'].apply(ast.literal_eval)
    df['End_point'] = df['End_point'].apply(ast.literal_eval)
    
    for _, row in df.iterrows():
        road_name = row['Road Name']
        start_lat,start_lng = row['Start_point']
        end_lat,end_lng = row['End_point']
        
        traffic_data = get_traffic_data_from_dataframe(gmaps_client, start_lat, start_lng, end_lat, end_lng, road_name)
        
        if traffic_data:
            traffic_data_list.append(traffic_data)

    if traffic_data_list:
        traffic_df = pd.DataFrame(traffic_data_list)
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        unique_filename = f"traffic_data_{timestamp}_{uuid.uuid4()}.csv"
        
        traffic_df.to_csv(unique_filename, index=False)
        
        print(f"Traffic data saved as '{unique_filename}'.")

generate_data_from_dataframe('filtered_road_names.csv', gmaps)


Traffic data saved as 'traffic_data_20240918_171831_fad6e4df-f7b7-423a-bee6-560c7a190ef3.csv'.
